# `scipy.stats` 통계 검정 라이브러리 정리
- 검정 결과마다 통계량이나 p-value 출력되는 순서나 형태 다를 수 있으므로, 헷갈리면 함수만 먼저 실행해보자
- 각각 값들을 반올림해서 출력할 경우도 있으므로, 함수만 실행 후 결과보고 객체 할당해주자

```python
# p-value 소수점 자리 반올림 : `format()`
print(format(p, '.6f'))
```

## 1. 일표본

### 단일표본(일표본) z검정
- 따로 라이브러리가 없어서 직접 함수 구현

참고 : 누구나 파이썬 통계분석

In [ ]:
# data : 샘플 데이터
# mu_mean : 모평균
# p_var : 알려진 모분산

import scipy.stats

def z_test(data, mu_mean, p_var, alpha=0.05) :
    s_mean = np.mean(data)
    n = len(data)
    rv = stats.norm()
    interval = rv.interval(1-alpha)
    
    z = (s_mean - mu_mean) / np.sqrt(p_var/n)
    
    if interval[0] <= z <= interval[1] :
        print("귀무가설 채택")
    else : 
        print("귀무가설 기각")
        
    if z < 0 :
        p = rv.cdf(z) * 2
    else :
        p = (1 - rv.cdf(z)) * 2
    print(f "p값은 {p : .3f}") 

In [ ]:
z_test(data, 130, 9) # 예시

### 단일표본(일표본) t검정 : `ttest_1samp()`

In [ ]:
from scipy.stats import ttest_1samp
s, p = ttest_1samp(data, 주장하는 모수)

### 일표본 윌콕슨 부호순위검정 : `wilcoxon()`
- 단일표본 t검정에 대응하는 비모수 검정법

In [ ]:
from scipy.stats import wilcoxon
s, p = wilcoxon(data-주장하는 모수) # 이렇게 빼줘야함

---

## 2. 이표본

### 정규성 검정 : `shapiro()`
- 모수 검정, 비모수 검정을 결정하기 위한 방법

#### 정규성 검정의 가설
- 귀무가설 : 데이터가 정규성을 따른다. (정규성 만족)
    - 표본의 모집단이 정규분포와 차이가 없다.
- 대립가설 : 데이터가 정규성을 따르지 않는다. (정규성 불만족)
    - 표본의 모집단이 정규분포와 차이가 있다.

In [ ]:
from scipy.stats import shapiro
s, p = shapiro(data1) # 각 데이터에 대해 실행

### 정규성을 만족 X, 등분산성 검정 : `levene()`
#### 등분산 검정의 가설
- 귀무가설 : 모든 그룹들의 분산은 동일하다.
- 대립가설 : 그룹들 중 적어도 한 그룹의 모분산이 동일하지 않다.

In [ ]:
from scipy.stats import levene
levene(data1, data2)

### 정규성을 만족 O, 등분산성 검정 : `bartlett()`

In [ ]:
from scipy.stats import bartlett
s, p = bartlett(data1, data2)

### Levene vs. Bartlett (△)
- Levene 검정은 그룹 내 오차의 분포를 정규분포 형태로 가정하지 않는다. 
    - 따라서, 정규성을 따르지 않는 데이터에 대해서도 상대적으로 강건한 검정 방법이다.
- Bartlett 검정은 그룹 내 오차의 분포를 정규분포로 가정한다.
    - 따라서, 정규성 가정을 만족하는 경우에 적용할 수 있다.
    - 그룹 간 분산의 비교에 있어 두 개 이상의 그룹에 대해서 적용할 수 있지만, 그룹 간 샘플 크기가 동일한 경우에 적합하다.

### 독립표본 t검정 : `ttest_ind()`
- 독립표본 t검정의 가정 : 정규성, 등분산성

-> 정규성 검정, 등분산성 검정은 독립표본 t검정에 필요한 검정들

#### 등분산성을 만족하지 않는 독립표본 t 검정? `equal_var = False` 옵션 추가 !

In [ ]:
from scipy.stats import ttest_ind
ttest_ind(data1, data2)

### 대응표본 t검정 : `ttest_rel()`
- 단측검정 시, 대립가설 방향 & 입력하는 변수 순서 주의

In [ ]:
from scipy.stats import ttest_rel

s, p = ttest_rel(df["후 데이터"], 
                 df["전 데이터"], 
                 alternative = "less") # 대립가설이 0보다 작다면 : less (후-전<0, 후<전)

---

## 3. 세 표본 이상

### 일원분산분석 : `f_oneway()`

In [ ]:
from scipy.stats import f_oneway
s, p = f_oneway(data1, data2, data3)

---

## 4. 범주형 변수 : 카이제곱검정

### 적합성 검정 (Goodness of fit test) : `chisquare()`
- 다항 모집단 비율의 차이
- 표본의 빈도를 이용하여 모집단의 가정된 빈도나 가정된 비율을 만족하는지를 확인하는 것
- 관찰빈도와 기대빈도의 차이를 이용한 검정
    - 관찰빈도는 `value_counts()` 관련

#### 적합성 검정의 가설
- 귀무가설 : ~의 비율이 2:1:1 '이다.'
    - ex) 실험에서 A,B,C 그룹간의 인원 수 비율은 2:1:1이다.
- 대립가설 :  ~의 비율이 2:1:1 '이 아니다.'
    - ex) 실험에서 A,B,C 그룹간의 인원 수 비율은 2:1:1이 아니다.

In [ ]:
from scipy.stats import chisquare

chisquare(df["observed"], df["expected"])

### 독립성 검정 : `chi2_contingency()`

#### 독립성 검정의 가설
- 귀무가설 : 두 변수는 독립이다. (연관이 없다.)
- 대립가설 : 두 변수는 독립이 아니다. (연관이 있다.) 

In [ ]:
from scipy.stats import chi2_contingency

# 교차표 생성
table = pd.crosstab(df["칼럼1"], df["칼럼2"])

# 카이제곱 독립성 검정
s, p, dof, expected = chi2_contingency(table)

#### 특정 기대도수를 구할 때 ? : `expected[찾는행-1, 찾는열-1]`
- 파이썬은 0부터 시작하므로
- 예를 들어, 교차표 3행 2열에 대응하는 기대도수를 찾으려면 `expected[2,1]` 이렇게 찾아야함

### 피셔의 정확검정 : `fisher_exact()`
- 데이터셋의 크기가 작고, 기대되는 빈도가 작을 때 피셔의 정확검정을 사용
- 비모수적인 방법으로서, 적은 샘플 크기에서도 정확한 결과를 제공
- 빈도수가 5개 이하 셀의 수가 전체 셀의 20%이상일 경우 사용

#### Example
- 귀무가설 : A와 B의 실력은 차이가 없다.
- 대립가설 : A의 실력이 더 좋다.

In [1]:
import pandas as pd
from scipy.stats import fisher_exact

table = pd.DataFrame([[8, 3], [4, 5]], index = ["A", "B"], columns=["승", "패"])
print(table)

s, p = fisher_exact(table, alternative="greater")

print()
print("검정통계량 : ", round(s, 3))
print("p-value : ", format(p, '.3f'))

   승  패
A  8  3
B  4  5

검정통계량 :  3.333
p-value :  0.205


---

## 5. 피어슨 상관계수 검정 : `pearsonr()`

### 상관계수 검정의 가설
$\rho$ : 두 변수의 모상관계수
- 귀무가설 : 두 변수 간 상관계수가 0이다. ($H_0 : \rho = 0$)
- 대립가설 : 두 변수 간 상관관수가 0이 아니다. ($H_0 : \rho \neq 0$)

In [ ]:
from scipy.stats import pearsonr 

rho, p = pearsonr(df["칼럼1"], df["칼럼2"])

## 6. 일반 상관계수 : `df.corr()`

### Example
iris에서 Sepal Length와 Sepal Width의 상관계수 계산하고 소수 둘째자리까지 출력하시오

[참고 : 캐글 링크](https://www.kaggle.com/code/agileteam/t3-correlation/notebook)

In [3]:
import pandas as pd
from sklearn.datasets import load_iris

# iris 데이터셋 로드
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [5]:
# Sepal Length와 Sepal Width의 상관계수 계산
df.corr()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
sepal length (cm),1.000000,-0.117570,0.871754,0.817941
sepal width (cm),-0.117570,1.000000,-0.428440,-0.366126
petal length (cm),0.871754,-0.428440,1.000000,0.962865
petal width (cm),0.817941,-0.366126,0.962865,1.000000


In [8]:
correlation = df.corr()
result = correlation.loc['sepal length (cm)', 'sepal width (cm)'] # loc로 특정 칼럼들에 대한 상관계수 출력
print(round(result, 2))

-0.12
